# Libraries 

In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 538.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 488.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.5 MB/s eta 0:00:0000:0100:01


In [2]:
import pyspark
import boto3
from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql import SparkSession


# Spark

In [3]:
conf = pyspark.SparkConf()

conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", 'oFhhQYp82kgEJfa6') \
    .set("spark.hadoop.fs.s3a.secret.key", 'a17pIbkBn63FY4XWgWOtrsY7gjnRUMZR') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

# MinIO 

In [4]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='oFhhQYp82kgEJfa6',
    aws_secret_access_key='a17pIbkBn63FY4XWgWOtrsY7gjnRUMZR',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

# Loading files

In [22]:
estados = spark.read.option("header", True).csv("./data/estados.csv")
municipios = spark.read.option("header", True).csv("./data/municipios.csv")

In [30]:
estados.show(5)

+---------+---+--------+--------+---------+------+
|codigo_uf| uf|    nome|latitude|longitude|regiao|
+---------+---+--------+--------+---------+------+
|       11| RO|Rondônia|  -10.83|   -63.34| Norte|
|       12| AC|    Acre|   -8.77|   -70.55| Norte|
|       13| AM|Amazonas|   -3.47|    -65.1| Norte|
|       14| RR| Roraima|    1.99|   -61.33| Norte|
|       15| PA|    Pará|   -3.79|   -52.48| Norte|
+---------+---+--------+--------+---------+------+
only showing top 5 rows



In [37]:
municipios.show(5)

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|codigo_ibge|               nome|latitude|longitude|capital|codigo_uf|siafi_id|ddd|     fuso_horario|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
|    5200050|    Abadia de Goiás|-16.7573| -49.4412|      0|       52|    1050| 62|America/Sao_Paulo|
|    3100104|Abadia dos Dourados|-18.4831| -47.3916|      0|       31|    4001| 34|America/Sao_Paulo|
|    5200100|          Abadiânia| -16.197| -48.7057|      0|       52|    9201| 62|America/Sao_Paulo|
|    3100203|             Abaeté|-19.1551| -45.4444|      0|       31|    4003| 37|America/Sao_Paulo|
|    1500107|         Abaetetuba|-1.72183| -48.8788|      0|       15|    0401| 91|America/Sao_Paulo|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+
only showing top 5 rows



# Send files

## parquet 

In [7]:
estados.write.save("s3a://aula-08/estados_parquet")

In [8]:
municipios.write.save("s3a://aula-08/municipios_parquet")

## csv 

In [46]:
estados.write.csv("./estados_csv")

In [47]:
municipios.write.csv("./municipios_csv")

## json 

In [48]:
estados.write.json("./estados_json")

In [49]:
municipios.write.json("./municipios_json")